 # Opis notatnika

 Ten notatnik inicjuje naszą pracę nad warsztatem końcowym. Naszym zadaniem tutaj jest pobranie udostępnionych nam danych do obszaru roboczego, które w następnym kroku wgramy na naszą bazę danych. Ich obróbka oraz analiza zostanie przeprowadzona w specjalnie do tego celu przygotowanych kolejnych notatnikach.

 Na potrzeby tego warsztatu został stworzony dedykowany serwis API, który dostępny jest pod adresem: https://api-datalab.coderslab.com/api/v2. Dodatkowo udostępniona została dokumentacja, z którą można zapoznać się tutaj: [klik](https://api-datalab.coderslab.com/v2/docs/).

 > Dokumentacja jest czysto techniczna i ma na celu prezentację dostępnych endpointów wraz ze zwracanym typem. W celu przetestowania należy kliknąć przysisk `Authorize`, podać token (dostępny poniżej), a następnie `Try it out!` oraz uzupełnić wymagane pola (parametry requesta).

 Zgodnie z dokumentacją stwierdzamy, że udostępnione zostały nam 4 endpointy:
 - `airport` - dane o lotnisku,
 - `weather` - informacje o zarejestrowaniej pogodzie na lotnisku danego dnia,
 - `aircraft` - dane o samolotach
 - `flights` - dane o wylotach z danego lotniska per dzień.

 Wszystkie te źródła musimy pobrać, aby być w stanie wykonać całość warsztatu. W celu pobrania informacji, gdzie wymagany jest paramatr `airportId`, posłużymy się listą z pliku `airports.csv`.



 ### Uwagi
 - Ze względów ćwiczeniowych, konstrukcja poszczególnych endpointów jest różna – w trakcie pracy dokładnie przyjrzyj się, w jaki sposób należy wykonać zapytanie, aby otrzymać odpowiedź.
 - Pamiętaj o dodaniu `sleep` pomiędzy poszczególnymi wywołaniami endpoint.
 - Limit wywołań API to 1000/min, zadbaj o nieprzekroczenie tego limitu – w przeciwnym wypadku będzie zwracany błąd 429.

 # Konfiguracja notatnika

 Tutaj zaimportuj wymagane biblioteki

In [1]:
import pandas as pd
import requests
import datetime
from datetime import datetime
import time
import os
from dateutil.relativedelta import relativedelta

 Tutaj zdefiniuj paramatry połączenia do API

In [2]:
api_url = 'https://api-datalab.coderslab.com/api/v2'
APIKEY = ''
headers = {'Authorization': f'{APIKEY}'}

 Tutaj wczytaj plik `airports.csv` i dostosuj do dalszych kroków w celu pobierania z kolejnych endpointów. Lista lotnisk jest dostępna w kolumnie `origin_airport_id`.

In [3]:
airports_df = pd.read_csv("../data/airports.csv")
airports_df.head()

,origin_airport_id
0,10874
1,11233
2,13360
3,15008
4,11638


 # Pobieranie `Airport`
 Zapoznaj się z dokumentacją endpointu `airport`, a następnie pobierz dane dot. poszczególnych lotnisk. Wyniki tego kroku zapisz do ramki `airport_df`, a następnie zapisz do pliku `csv`.

 ### Wskazówki
 - Nie wszystkie lotniska dostępne w pliku `airports.csv`, są dostępne w endpoint. Zadbaj o odpowiednie obsłużenie takiej sytuacji,
 - Do skonwertowania wyników przydatna może okazać się metoda `Pandas` - [from_records](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html),
 - Artykuł LMS: `Python - analiza danych > Dzień 4 - API > Uwierzytelnianie`
 - Artykuł LMS: `Python - analiza danych > Przygotowanie do zjazdu 2`

 Tutaj pobierz dane z endpoint'u `airport`

In [4]:
airport_data_list = []

for index, row in airports_df.iterrows():
    airport_id = row['origin_airport_id']
    response = requests.get(f'{api_url}/airport/{airport_id}', headers=headers)
    
    if response.status_code == 401:
        print('Niepoprawny klucz, lub po prostu jeszcze nie aktywny')
    elif response.status_code == 200:
        airport_data_list.append(response.json())

In [5]:
airport_df = pd.DataFrame.from_records(airport_data_list)

airport_df.head()

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
0,11638,Fresno Air Terminal,"Fresno, CA","FRESNO YOSEMITE INTERNATIONAL, CA US"
1,13342,General Mitchell Field,"Milwaukee, WI","MILWAUKEE MITCHELL AIRPORT, WI US"
2,13244,Memphis International,"Memphis, TN","MEMPHIS INTERNATIONAL AIRPORT, TN US"
3,15096,Syracuse Hancock International,"Syracuse, NY","SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US"
4,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić czy ta część została poprawnie wykonana

In [6]:
airport_df_expected_shape = (97, 4)
assert airport_df_expected_shape == airport_df.shape

 Tutaj zapisz ramkę `airport_df` do pliku `airport_list.csv`

In [7]:
airport_df.to_csv("../data/raw/airport_list.csv", index=False)

 # Pobieranie `Weather`
 Zapoznaj się z dokumentacją endpotu `Weather`, następnie pobierz dane dotyczące zarejestrowanej pogody na poszczególnych lotniskach. Wyniki zapisz do ramki `weather_df`, a później do pliku `airport_weather.csv`.

 Wskazówki:
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`, czyli stworzenie listy, a następnie przekonwertowanie jej w postać ramki.
 - Data początkowa danych to `2019-01-01`, zaś data końcowa to `2020-03-31`, czyli 15 miesięcy,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.
 - Przy dodawaniu miesięcy do daty może przydać się metoda [relativedelta](https://www.geeksforgeeks.org/python-get-month-from-year-and-weekday/).

In [8]:
weather_data_list = []
start_date = datetime.strptime('2018-12-1', '%Y-%m-%d')
end_date = datetime.strptime('2020-03-29', '%Y-%m-%d')

current_date = start_date

while current_date <= end_date:
    
    current_date_str = current_date.strftime('%Y-%m-%d')
    current_date += relativedelta(months=1)
      
    response = requests.get(f'{api_url}/airportWeather?date={current_date_str}', headers=headers)
    
    if response.status_code == 401:
        print('Niepoprawny klucz, lub po prostu jeszcze nie aktywny')
    elif response.status_code == 200:
        relevant_columns = [
            "STATION", "NAME", "DATE", "AWND", "PGTM", "PRCP", "SNOW", "SNWD",
            "TAVG", "TMAX", "TMIN", "WDF2", "WDF5", "WSF2", "WSF5",
            "WT01", "WT02", "WT03", "WT04", "WT05", "WT06", "WT07", "WT08",
            "WT09", "WESD", "WT10", "PSUN", "TSUN", "SN32", "SX32", "TOBS",
            "WT11", "WT18"
        ]

        
        for item in response.json():
            weather_data_list.append({key: item.get(key, None) for key in relevant_columns})

In [9]:
airport_weather_df = pd.DataFrame(weather_data_list)
airport_weather_df.head(1)

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,...,WT09,WESD,WT10,PSUN,TSUN,SN32,SX32,TOBS,WT11,WT18
0,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-01,4.7,NaN,0.14,0.0,0.0,64.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [11]:
airport_weather_df_expected_shape = (46226, 33)
assert airport_weather_df_expected_shape == airport_weather_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `weather_df` do pliku `airport_weather.csv` w katalogu `data/raw`

In [12]:
airport_weather_df.to_csv("../data/raw/airport_weather.csv", index=False)

 # Pobranie `Aircraft`
 Zapoznaj się z dokumentacją endpointu `aircraft`, a następnie pobierz dane produkcyjne samolotów. Wyniki zapisz do ramki `aircraft_df`, a następnie zapisz do pliku `aircraft.csv`.


In [13]:
aircraft_data_list = []

response = requests.get(f'{api_url}/aircraft', headers=headers)

if response.status_code == 200:
    aircraft_data_list = response.json()

In [14]:
aircraft_df = pd.DataFrame(aircraft_data_list)
aircraft_df.head(1)

,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
0,1944,N54514,0.0


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [15]:
aircraft_df_expected_shape = (7383, 3)
assert aircraft_df_expected_shape == aircraft_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `aircraft_df` do pliku `aircraft.csv` w katalogu `data/raw`

In [16]:
aircraft_df.to_csv("../data/raw/aircraft.csv", index=False)

 # Pobranie `Flight`
 Zapoznaj się z dokumentacją endpointu `flights`, następnie pobierz dane dotyczące ruchu lotniczego. Wyniki zapisz do ramki `flight_df`, a później do pliku `flight.csv`.

 Wskazówki:
 - Zwróć szczególną uwagę na konstrukcję endpointa,
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego, z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`,
 - Data początkowa danych to `2019-01-01`, zaś końcowa to `2020-03-31`, czyli 456 dni,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku,
 - W przypadku, gdy nie ma dostępnych danych dla danego lotniska, API zwraca kod [204](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/204), w ten sposób możesz pominąć lotniska, dla których dane nie są dostępne,
 - Pobranie całości danych zajmuje dłuższą chwilę, zanim włączysz pętle dla wszystkich danych, sprawdź pobieranie danych dla jednego, dwóch lotnisk aby uniknąć frustracji.

In [ ]:
start_date = datetime.strptime('2019-1-1', '%Y-%m-%d')
end_date = datetime.strptime('2020-03-29', '%Y-%m-%d')

flight_data = []
for index, row in airports_df.iterrows():
    airport_id = row['origin_airport_id']
    current_date = start_date  

    while current_date <= end_date:
        current_date_str = current_date.strftime('%Y-%m-%d')
        current_date += relativedelta(months=1)

        response = requests.get(f'{api_url}/flight?airportId={airport_id}&date={current_date_str}', headers=headers)

        if response.status_code == 200:
            flight_data.extend(response.json())
            
        elif response.status_code == 204:
            break
        else:
            print(f"Błąd pobierania danych dla lotniska {airport_id}. Status code: {response.status_code}")
            
    if (index + 1) % 10 == 0:
        print(f'Processed {index + 1} airports.')


flight_df = pd.DataFrame(flight_data)

Processed 10 airports.
Processed 20 airports.
Processed 30 airports.
Processed 40 airports.
Processed 50 airports.
Processed 60 airports.
Processed 70 airports.
Processed 80 airports.
Processed 90 airports.
Processed 100 airports.
Processed 110 airports.
Processed 120 airports.
Processed 130 airports.


In [ ]:
flight_df.head()

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
flight_df_expected_shape = (1386120, 27)
assert flight_df_expected_shape == flight_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `flight_df` do pliku `flight.csv` w katalogu `data/raw`

In [ ]:
flight_df.to_csv("../data/raw/flight.csv", index=False)

 # Podsumowanie
 W tym notatniku wykonaliśmy podstawowy krok w analizie danych - pozyskaliśmy je. Są gotowe do dalszej pracy, czyli możemy załadować je na bazę danych, a następnie zapoznać się z tym, jakie informacje ze sobą niosą. Kolejne notatniki będą służyły właśnie tym celom.

In [ ]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)